# Wave Function

Lars Schuster 10.11.2022

The wave function can be numerically solved by discretization in space $(h)$ and time $({\delta}_t)$.
In this case a string with two fixed ends is considered.

To do that the leapfrog algorithm is used.
The algorithm calculates the next step in time by taking into account the two previous steps in time. 
For this not only the same position is considered but also adjacent positions on the string. 

A single point on the string is denoted as $u_i^j$ where $j$ is the index for the time $(t = j{\delta}_t)$ and $i$ the index for the postion in space $(x = ih)$.

The formular is given by:

$u_i^{j+1} = 2u_i^j - u_i^{j-1} + \frac{c^2}{{c'}^2}(u_{i+1}^j + u_{i-1}^j)$

To increase efficiency all values are updated at once with a matrix that is composed of multiple diagonal matrices.

The algorithm has an initial value problem because usually only one inital state will be given and the first time derivative at this time. To solve it the first_step function is implemented assuming that the time derivative is 0. 
(This is the case if the string is being plucked and released.)

The first_step function takes the initial state u1 , the wave-velocity c, the lattice velocity c_dash and the length of the array len as parameters.

With the following formular the first step is calculated form the initial state and the algorithm can start afterwards.

$u_i^{1} =  \frac{c^2}{{c'}^2}(u_{i+1}^0 + u_{i-1}^0) +(1- \frac{c^2}{{c'}^2})u_i^0+  \delta_t \frac{\partial u(x_i,0)}{\partial t}|_{t=0}$

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

def first_step(u1, c, c_dash, len):
    u_new = np.zeros(int(len))

    d1 = np.diag(np.ones(len, dtype=float)  - (c/c_dash)**2)
    d2 = np.diag(np.ones(len-1, dtype=float) * 0.5 *(c/c_dash)**2, 1)
    d3 = np.diag(np.ones(len-1, dtype=float) * 0.5 *(c/c_dash)**2, -1)

    m1 = d1 + d2 + d3
    u_new = m1 @ u1
    u_new[0] = 0
    u_new[int(len)-1] = 0
    return u_new

The wave function completes the entire computations for all iterations of the wave funciton.

The parameters are the inital state i_state, the veloctiy of the wave c, the time discretization d_t, the space discretization, the length of the string l and the number of iterations it.

The length of the array is calculated by the quotient of the length and the lattice spacing h.
The lattice velocity c_dash is computed with the quotient of the lattice spacing h and the timestep d_t
Firstly the first iteration is calculated with the fist_step funciton.

In [23]:
def wave(i_state, c, d_t, h, l, it):
    """
    Parameters:
        i_state: a numpyarray symbolizing the initial state
        c: a float that is the velocity
        d_t: a float that is the time step
        h: a float that is the space step
    """
    #initialization
    arr_len = int(l/h)
    c_dash = h / d_t
    result = np.zeros((it, int(arr_len)))
    result[0] = i_state
    result[0, 0] = 0
    result[0, int(arr_len)-1] = 0
    result[1] = first_step(i_state, c, c_dash, arr_len)

    #create diagonal matrix with 2 -2(c/c_dash)**2 on the diagonal
    d1 = np.diag(np.ones(arr_len, dtype=float) * 2 - 2*(c/c_dash)**2)
    d2 = np.diag(np.ones(arr_len-1, dtype=float) * (c/c_dash)**2, 1)
    d3 = np.diag(np.ones(arr_len-1, dtype=float) * (c/c_dash)**2, -1)
    m1 = d1 + d2 + d3

    for i in range(2, it):
        result[i] = m1 @ result[i-1] - result[i-2]
        result[i, 0] = 0
        result[i, int(arr_len)-1] = 0
    return result


The plot function will create a plot of a singular state of the wave.

The animate funciton will animate the entire data of the wave.

In [24]:
#function to plot one iteration
def plot(x, y):
    plt.title("Postion as a graph of time")
    plt.xlabel('time in s')
    plt.ylabel('Postion in m')
    plt.plot(x, y)
    plt.ylim(-1.05, 1.05)
    plt.grid()
    plt.tight_layout()
    plt.show()
    plt.close

from IPython import display
#animate the entire data
def animate(data, it):
    x = np.linspace(0,1,(data[0].size))
    fig,ax = plt.subplots()
    ax.set_xlim(( 0, 1))
    ax.set_ylim((-2.05, 2.05))
    ax.set_title("Wave in a String")
    ax.set_xlabel("x along string in m")
    ax.set_ylabel("y-deflection in m")
    wave, = ax.plot(x,data[0],"-")
    
    def anim(i):
        wave.set_ydata(data[i])
        return wave,
    
    anim = animation.FuncAnimation(fig, anim, frames=it, interval=20)
    writer = animation.writers['html']
    video = anim.to_html5_video()
    html = display.HTML(video)
    display.display(html)

    plt.close()



To show the results of the code imagine a string of length 1m which is plucked in the middle. 

To simulate this the inital state is given by 

$u(x) = e^{-200(x-0.5)^2}$

The lattice spacing (0.01m) and the discretezation in time(0.0001s) are chosen so that the lattice velocity is equal to the wave velocity. (in this case $100 \frac{m}{s}$)

In [26]:
x = np.linspace(0,1,100)

def y_func(x):
    a = np.exp(-200.* (x-0.50)**2)
    return a

i_state = y_func(x)
c = 100 #in m/s
d_t = 0.0001 #in seconds
h = 0.01 #in m
len = 1 #in m

it = 200
data = wave(i_state,c,d_t,h,len,it)

animate(data,it)

This result is stable and actually aggres with the analytical solution.

If the time step is decreased the result is still stable:

In [28]:
x = np.linspace(0, 1, 100)
i_state = y_func(x)
c = 100 #in m/s
d_t = 0.00005 #in seconds
h = 0.01 #in m
len = 1 #in m

it = 200
data = wave(i_state,c,d_t,h,len,it)

animate(data,it)

However if the lattice spacing is reduced to half the original value and the time step is again 0.0001s the model is not stable anymore and results in very odd outputs.

In [31]:
x = np.linspace(0, 1, 200)
i_state = y_func(x)
c = 100 #in m/s
d_t = 0.0001 #in seconds
h = 0.005 #in m
len = 1 #in m

it = 200
data = wave(i_state,c,d_t,h,len,it)

animate(data,it)

However, if both parameters are halved the simulation is stable again:

In [32]:
x = np.linspace(0, 1, 200)
i_state = y_func(x)
c = 100 #in m/s
d_t = 0.00005 #in seconds
h = 0.005 #in m
len = 1 #in m

it = 200
data = wave(i_state,c,d_t,h,len,it)

animate(data,it)

It is also interestig to consider other cases such as a string plucked closer to one end:

In [35]:
x = np.linspace(0,1,100)

def y_func(x):
    a = np.exp(-200.* (x-0.86)**2)
    return a

i_state = y_func(x)
c = 100 #in m/s
d_t = 0.0001 #in seconds
h = 0.01 #in m
len = 1 #in m

it = 200
data = wave(i_state,c,d_t,h,len,it)

animate(data,it)

If the string is plucked at the very boundary it will take on a rather odd shape:

In [36]:
x = np.linspace(0,1,100)

def y_func(x):
    a = np.exp(-200.* (x-1)**2)
    return a

i_state = y_func(x)
c = 100 #in m/s
d_t = 0.0001 #in seconds
h = 0.01 #in m
len = 1 #in m

it = 200
data = wave(i_state,c,d_t,h,len,it)

animate(data,it)

The pulse travels back and forth between the ends.